In [1]:
# import dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
import statistics

# import config
from config import league_id, api_key_MFL

In [2]:
# Get name, team name, position
urlString = f'https://api.myfantasyleague.com/2021/export?TYPE=players'
response_players = requests.get(urlString)
soup_players = BeautifulSoup(response_players.content,'xml')

player_data = []
players = soup_players.find_all('player')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("name"), players[i].get("position"), players[i].get("team")]
    player_data.append(rows)
player_df = pd.DataFrame(player_data)
player_df.columns=['PlayerID','Name', 'Position', 'Team']



In [3]:
# Get Total Scores
urlString = f'https://api.myfantasyleague.com/2021/export?TYPE=playerScores&L={league_id}&W=YTD'
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('playerScore')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("score")]
    data.append(rows)
totalScores = pd.DataFrame(data)
totalScores.columns=['PlayerID','Score']
totalScores['Score'] = totalScores['Score'].astype('float64')


In [4]:
# Get Average Score
urlString = f"https://api.myfantasyleague.com/2021/export?TYPE=playerScores&L={league_id}&W=AVG"
response = requests.get(urlString)
soup = BeautifulSoup(response.content,'xml')

data = []
players = soup.find_all('playerScore')
for i in range(len(players)):
    rows = [players[i].get("id"), players[i].get("score")]
    data.append(rows)
averageScore = pd.DataFrame(data)
averageScore.columns=['PlayerID','ExtrapolatedAverage']
averageScore['ExtrapolatedAverage'] = averageScore['ExtrapolatedAverage'].astype('float32')

list = []
for i in range(len(averageScore)):
    extrapolation = (averageScore.iloc[i]['ExtrapolatedAverage'])*16
    list.append(extrapolation)
averageScore['ExtrapolatedAverage'] = list

In [5]:
# Merge dfs
playered = pd.merge(player_df, totalScores, on='PlayerID')
playered = pd.merge(playered, averageScore, on='PlayerID')
playered = playered.sort_values(by='Score', ascending=False)

In [6]:
# Calculate Relative value for each df
qbs = playered[playered['Position'] == "QB"].reset_index(drop=True)
rbs = playered[playered['Position'] == "RB"].reset_index(drop=True)
wrs = playered[playered['Position'] == "WR"].reset_index(drop=True)
tes = playered[playered['Position'] == "TE"].reset_index(drop=True)
pks = playered[playered['Position'] == "PK"].reset_index(drop=True)
defs = playered[playered['Position'] == "Def"].reset_index(drop=True)

df_list = [qbs, rbs, wrs, tes, pks, defs]
playing = [24, 36, 48, 24, 24, 24]
for i in range(len(df_list)):
    df = df_list[i]
    floor_spot = playing[i]
    position_floor = df.iloc[floor_spot]['Score']
    df['Floor'] = position_floor
    df['RelativeScore'] = df['Score'] - df['Floor']
    df['RelativeAverage'] = df['ExtrapolatedAverage'] - df['Floor']
    
    relList = []
    for j in range(len(df)):
        rel = statistics.mean([df.iloc[j]['RelativeScore'], df.iloc[j]['RelativeAverage']])
        relList.append(rel)
    df['Relative'] = relList
    df.sort_values(by='Relative', ascending=False, inplace=True)
    
    regList = []
    for k in range(len(df)):
        if k == 0:
            rel1 = df.iloc[(k)]['Relative']
        else:
            rel1 = df.iloc[(k-1)]['Relative']
        rel2 = df.iloc[(k)]['Relative']
        if k == (len(df)-1):
            rel3 = df.iloc[(k)]['Relative']
        else:
            rel3 = df.iloc[(k+1)]['Relative']
        reg = statistics.mean([rel1, rel2, rel3])
        regList.append(reg)
    df['Regressed'] = regList
    df.reset_index(inplace=True, drop=True)

In [7]:
tes

,PlayerID,Name,Position,Team,Score,ExtrapolatedAverage,Floor,RelativeScore,RelativeAverage,Relative,Regressed
0,13671,"Andrews, Mark",TE,BAL,210.8,210.720001,59.7,151.1,151.020001,151.060001,144.193335
1,11244,"Kelce, Travis",TE,KCC,184.0,196.320007,59.7,124.3,136.620007,130.460004,127.150002
2,13299,"Kittle, George",TE,SFO,143.1,176.160004,59.7,83.4,116.460004,99.930002,103.826669
3,9902,"Gronkowski, Rob",TE,TBB,114.7,166.880005,59.7,55.0,107.180005,81.090002,85.670000
4,13772,"Schultz, Dalton",TE,DAL,135.7,135.679993,59.7,76.0,75.979993,75.989996,75.899998
...,...,...,...,...,...,...,...,...,...,...,...
140,15506,"Bushman, Matt",TE,KCC,0.0,0.000000,59.7,-59.7,-59.700000,-59.700000,-59.700000
141,15593,"Ralston, Nick",TE,DAL,0.0,0.000000,59.7,-59.7,-59.700000,-59.700000,-59.700000
142,15534,"Forristall, Miller",TE,CLE,0.0,0.000000,59.7,-59.7,-59.700000,-59.700000,-59.700000
143,15542,"Bowers, Nick",TE,LVR,0.0,0.000000,59.7,-59.7,-59.700000,-59.700000,-59.700000


In [9]:
with pd.ExcelWriter("../data/RelativeValues.xlsx") as writer:
    qbs.to_excel(writer, sheet_name='QB')
    rbs.to_excel(writer, sheet_name='RB')
    wrs.to_excel(writer, sheet_name='WR')
    tes.to_excel(writer, sheet_name='TE')
    pks.to_excel(writer, sheet_name='PK')
    defs.to_excel(writer, sheet_name='DEF')